### 1 导入数据

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_ = pd.read_csv(r'./附件2_num.csv')
df = df_.iloc[:18060,:]

In [3]:
df_ = pd.read_csv(r'./附件2_num.csv')
df = df_.iloc[:18060,:]
df_indus = pd.read_excel(r'./附件1.xlsx')
df = pd.merge(df,df_indus,on="TICKER_SYMBOL",how='left')
#提取制造业分析
df_manu=df[df['所属行业']=='制造业']
#df_manu = pd.read_csv(r'./制造业_full.csv')#利用Excel将制造业数据划分

D:\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


### 2 数据预处理
#### 2.1 移除缺失大的特征

In [4]:
def find_missing(data):
    missing_num =data.isna().sum(axis=0).sort_values(ascending=False)
    missing_prop=missing_num/float(len(data))
    drop_index=missing_prop[missing_prop>0.80].index.tolist()#筛选出缺失值大于50%的特征
    return drop_index

df_manu = df_manu.drop(find_missing(df_manu.iloc[:,1:-2]),axis=1)

#### 2.2 移除低方差特征

In [5]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=1)
df1 = pd.DataFrame(sel.fit_transform(df_manu.iloc[:,:-2]))
print(df_manu.shape,df1.shape)
df1.columns = df_manu.iloc[:,:-2].columns[sel.get_support(indices=False)]
df1.head()

(9943, 268) (9943, 261)


,TICKER_SYMBOL,ACT_PUBTIME,PUBLISH_DATE,END_DATE_REP,END_DATE,CASH_C_EQUIV,NOTES_RECEIV,AR,PREPAYMENT,INT_RECEIV,...,DAYS_AR,AP_TURNOVER,CA_TURNOVER,OPER_CYCLE,INVEN_TURNOVER,FA_TURNOVER,TFA_TURNOVER,DAYS_AP,DAYS_INVEN,AR_TURNOVER
0,4019.0,3.0,3.0,2.0,1.0,6.719939e+08,1.643520e+08,3.451945e+08,1.039522e+07,2311657.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8166.0,3.0,3.0,2.0,1.0,1.010981e+09,2.155752e+09,2.887470e+08,1.075202e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11737.0,3.0,3.0,2.0,1.0,1.502023e+09,2.688988e+07,9.517806e+08,4.157376e+07,6464811.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16479.0,3.0,3.0,2.0,1.0,9.966031e+08,3.686740e+05,2.824134e+08,2.741431e+07,2528572.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16842.0,4.0,4.0,3.0,1.0,5.763404e+07,8.990000e+06,5.941825e+07,3.603420e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2.3 基于KNN模型的缺失值填补

In [6]:
from sklearn.impute import KNNImputer

imputer=KNNImputer()
df1_full = pd.DataFrame(imputer.fit_transform(df1,df_manu['FLAG']))
print(sum(df_manu.iloc[:,-2]))
df1_full.columns = df_manu.iloc[:,:-2].columns[sel.get_support(indices=False)]
df1_full.head()

80.0


,TICKER_SYMBOL,ACT_PUBTIME,PUBLISH_DATE,END_DATE_REP,END_DATE,CASH_C_EQUIV,NOTES_RECEIV,AR,PREPAYMENT,INT_RECEIV,...,DAYS_AR,AP_TURNOVER,CA_TURNOVER,OPER_CYCLE,INVEN_TURNOVER,FA_TURNOVER,TFA_TURNOVER,DAYS_AP,DAYS_INVEN,AR_TURNOVER
0,4019.0,3.0,3.0,2.0,1.0,6.719939e+08,1.643520e+08,3.451945e+08,1.039522e+07,2311657.010,...,25.95774,7.92418,1.23638,194.73914,2.88350,1.56300,1.42796,51.28988,193.76240,17.56994
1,8166.0,3.0,3.0,2.0,1.0,1.010981e+09,2.155752e+09,2.887470e+08,1.075202e+08,3560974.464,...,66.86108,31.29172,2.18356,191.36818,7.79424,15.66504,16.38296,67.77566,124.50714,12.01438
2,11737.0,3.0,3.0,2.0,1.0,1.502023e+09,2.688988e+07,9.517806e+08,4.157376e+07,6464811.930,...,64.87798,9.48186,0.73224,143.15022,7.49298,9.11650,7.88524,58.12448,78.27224,6.59040
3,16479.0,3.0,3.0,2.0,1.0,9.966031e+08,3.686740e+05,2.824134e+08,2.741431e+07,2528572.870,...,42.59152,5.03148,1.40024,229.23174,4.21970,7.89938,7.77184,91.31484,186.64022,13.56754
4,16842.0,4.0,4.0,3.0,1.0,5.763404e+07,8.990000e+06,5.941825e+07,3.603420e+07,65976.256,...,120.13558,7.72594,1.38076,57.35044,34.91542,25.71770,26.89816,71.93672,77.27698,4.06470
